In [ ]:
import pandas as pd
import os

/Users/selimane/Documents/prescription_med/code/preprocessing


In [6]:
dfs_atc1 = []
dfs_atc2 = []

In [9]:
df_mapping = pd.read_csv("../../data/raw/OPEN_MEDIC_2024.csv", sep=";", encoding="latin1")

df_mapping.columns = df_mapping.columns.str.strip().str.lower()

mapping_atc1 = (df_mapping[["atc1", "l_atc1"]].dropna().drop_duplicates())
mapping_atc2 = (df_mapping[["atc2", "l_atc2"]].dropna().drop_duplicates())

In [ ]:
years = (2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014)


for year in years :
    path = BASE_DIR / "data" / "raw" / f"OPEN_MEDIC_{year}.csv"
    df_raw = pd.read_csv(path, sep = ";", encoding = "latin1")
    df_raw.columns = df_raw.columns.str.strip().str.lower()

    df_raw.rename(columns = {"ben_reg" : "region"}, inplace = True)
    df_raw["boites"] = pd.to_numeric(df_raw["boites"], errors = "coerce")
    df_raw["region"] = pd.to_numeric(df_raw["region"], errors = "coerce")
    df_raw = df_raw.dropna(subset = ["region", "boites"])

    df_raw.drop(columns = ["l_atc1", "l_atc2", "atc3","l_atc3","atc4","l_atc4","atc5","l_atc5","cip13","l_cip13","top_gen", "gen_num", "age","sexe","psp_spe","rem","bse"], inplace = True, errors = "ignore")
    df_raw = df_raw.merge(mapping_atc1, on = "atc1", how = "left")
    df_raw = df_raw.merge(mapping_atc2, on = "atc2", how = "left")
    
    df_raw["annee"] = year

    df_raw = df_raw[~df_raw["region"].isin([0,5,99])]

    df_atc1 = df_raw.groupby(["annee", "atc1", "l_atc1", "region"], as_index = False)["boites"].sum()
    df_atc2 = df_raw.groupby(["annee", "atc2", "l_atc2", "region"], as_index = False)["boites"].sum()

    del df_raw

    dfs_atc1.append(df_atc1)
    dfs_atc2.append(df_atc2)



In [11]:
df_atc1_all = pd.concat(dfs_atc1, ignore_index=True)
df_atc2_all = pd.concat(dfs_atc2, ignore_index=True)

In [13]:
df_atc1_all.to_csv(BASE_DIR / "data/processed/atc1_prescriptions_2014_2024.csv", sep=";", index=False)
df_atc2_all.to_csv(BASE_DIR / "data/processed/atc2_prescriptions_2014_2024.csv", sep=";", index=False)